In [1]:
date = '20220930'  #current date

import numpy as np
import pandas as pd
import pandasql
import sqlite3

pd.set_option("display.max_columns", None) 
pd.set_option("display.max_colwidth", 1000) #huruf dlm column
pd.set_option("display.max_rows", 100)
pd.set_option("display.precision", 2) #2 titik perpuluhan


In [2]:
#15 min run
Python = pd.read_csv(r"D:\Users\80005701\Desktop\IA Moodys VS DWH\Data\IA_"+str(date)+"(python) w acc.txt", sep = ",", header = 0, low_memory = False)
Moodys = pd.read_csv(r"D:\Users\80005701\Desktop\IA Moodys VS DWH\Data\Moodys"+str(date)+".txt", sep = ",", header = 0, low_memory = False)


#prev_date = 20220531
#location_cust = r"D:\Users\20220531\S001_004_ACRM_CUST"+str(prev_date)+".txt"

location_cust = r"D:\Users\\"+str(date)+"\\S001_004_ACRM_CUST"+str(date)+".txt"
cust = pd.read_csv(location_cust, sep = "|", header = 0, error_bad_lines = False)
cust.columns = cust.columns.str.replace(" ", "_")
cust.M_SUB_SUB_MARKET_SEGMENT.replace({'    ': -9999}, inplace=True)


In [4]:
#Moodys3  = pd.read_excel(r"D:\Users\80005701\Desktop\IA Moodys VS DWH\Data\Moodys"+str(date)+" (Overall Undrawn).xlsx")

#Moodys2  = pd.read_excel(r"D:\Users\80005701\Desktop\IA Moodys VS DWH\Data\Moodys"+str(date)+" (Summary All).xlsx", sheet_name='Context Id=272_1')

#Moodys1  = pd.read_excel(r"D:\Users\80005701\Desktop\IA Moodys VS DWH\Data\Moodys"+str(date)+" (Summary All).xlsx", sheet_name='Context Id=272_0')

#frames = [Moodys1, Moodys2, Moodys3] 
#Moodys = pd.concat(frames)

#Moodys.to_csv(r"D:\Users\80005701\Desktop\IA Moodys VS DWH\Data\Moodys"+str(date)+".txt", index = False)


In [4]:
Moodys['COUNTERPARTY_ORIGINAL'] = Moodys['COUNTERPARTY_ORIGINAL'].astype('Int64').astype('str')
Python['M_CUS_NO'] = Python['M_CUS_NO'].astype('Int64').astype('str')
cust['CUS_NO'] = cust['CUS_NO'].astype('Int64').astype('str')


In [101]:
#Python_Adhoc = Python.iloc[np.where(Python.F_RPSIA=='N')]

#CUST LEVEL
Python1 = Python.fillna(0).groupby(['M_CUS_NO'])[['M_BNM_BALANCE2','EAD_PROPORTIONAL_MYR','EXPECTED_LOSS_MYR','RWA_AFTER_RISK_MIT_MYR']].sum().reset_index()
Moodys1 = Moodys.fillna(0).groupby(['COUNTERPARTY_ORIGINAL'])[['SUM(OUTSTANDING)','EAD','EXPECTED_LOSS','RWA_AFTER_SCALING']].sum().reset_index()

#to cek same n diff
cust_1 = cust[["CUS_NO" ,"NAME"]]
cust_1['Teradata_Tag'] = 'Y'

Python2 = Python1.merge(Moodys1[['COUNTERPARTY_ORIGINAL','SUM(OUTSTANDING)','EAD','EXPECTED_LOSS','RWA_AFTER_SCALING']].rename(columns={'COUNTERPARTY_ORIGINAL':'M_CUS_NO'}), on='M_CUS_NO',how='left', indicator=True).\
merge(cust_1.rename(columns={"CUS_NO": "M_CUS_NO"}), on = 'M_CUS_NO', how = 'left')

Moodys2 = Moodys1.merge(Python1[['M_CUS_NO','M_BNM_BALANCE2','EAD_PROPORTIONAL_MYR','EXPECTED_LOSS_MYR','RWA_AFTER_RISK_MIT_MYR']].rename(columns={'M_CUS_NO':'COUNTERPARTY_ORIGINAL'}), on='COUNTERPARTY_ORIGINAL',how='left', indicator=True).\
merge(cust_1.rename(columns={"CUS_NO": "COUNTERPARTY_ORIGINAL"}), on = 'COUNTERPARTY_ORIGINAL', how = 'left')

D:\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [102]:
Python2._merge.value_counts() 

both          1673566
left_only       25514
right_only          0
Name: _merge, dtype: int64

In [103]:
Moodys2._merge.value_counts() 

both          1673566
left_only        6785
right_only          0
Name: _merge, dtype: int64

# Case: Mismatch (Overall)

In [104]:
#cek by account level
Python_left = Python2.iloc[np.where(Python2._merge=="left_only")][['M_CUS_NO','NAME','Teradata_Tag']].merge(Python, on='M_CUS_NO',how='left')
Moodys_left = Moodys2.iloc[np.where(Moodys2._merge=="left_only")][['COUNTERPARTY_ORIGINAL','NAME','Teradata_Tag']].merge(Moodys, on='COUNTERPARTY_ORIGINAL',how='left')


In [68]:
#export
writer = pd.ExcelWriter(r'D:\Users\80005701\Desktop\IA Moodys VS DWH\00. IA_ Left only.xlsx',engine='xlsxwriter')

a.iloc[np.where(a._merge=="left_only")].to_excel(writer, sheet_name='Python cust'+str(date), index = False)
Python_left.to_excel(writer, sheet_name='Python acc'+str(date), index = False)

b.iloc[np.where(b._merge=="left_only")].to_excel(writer, sheet_name='Moodys cust'+str(date), index = False)
Moodys_left.to_excel(writer, sheet_name='Moodys acc'+str(date), index = False)

writer.save()


# Case: Match (01. CB)

In [76]:
Python_both.PRODUCT_GROUP_APC.value_counts()

07. Hire Purchase             973946
08. Credit Cards              418054
09. Unit Trust                390885
04. Home Financing            241352
03. RSME Banking               62494
11. Salary                     40388
02. Business Banking           15944
05. Shophouse                  13089
01. Corporate Banking           8266
06. Other Mortgage              7111
13. Cashline                    4423
12. Other Retail Financing       335
14. SMF                          151
Name: PRODUCT_GROUP_APC, dtype: int64

In [115]:
#Cus level all both
Python_both = Python2.iloc[np.where(Python2._merge=='both')] #cukup satu sbb ad dedua value
#Moodys_both = Moodys2.iloc[np.where(Moodys2._merge=='both')]


#cek by account level
Python_Acc = Python_both[['M_CUS_NO','NAME','Teradata_Tag']].merge(Python, on='M_CUS_NO',how='left')


In [164]:
sum(Python_both.M_BNM_BALANCE2), sum(Python_Acc.M_BNM_BALANCE2)

(228292005367.3618, 228292005367.36746)

In [165]:
#cek by product
Python_prod = Python_Acc.iloc[np.where(Python_Acc.PRODUCT_GROUP_APC=='01. Corporate Banking')]

In [173]:
Python_prod.iloc[np.where(Python_prod.M_CUS_NO=='10013113')]

,M_CUS_NO,NAME,Teradata_Tag,DATA_SOURCE,M_ACCOUNT_NO,LEVEL_6_ORI,LEVEL_6,M_SUB_MARKET_SEGMENT_CD,M_PRODUCT_HIERARCHY_CD,M_SOURCE_PROD_TYPE_CD,SOURCE_PROD_TYPE_DSC,M_MTHS_IN_ARREARS,APPROACH,SECTOR_CATEGORY,PD_SEGMENT,RSME_INCLUSION_INDICATOR,RS_INCLUSION_INDICATOR,REPORT_INDICATOR,F_RPSIA,RPSIA_THIRD_PARTY,IMPAIRED_FLAG,F_NCIF_CUST,F_NCIF_ACCT,SECURITY_TYPE_CD,M_FINANCING_CONCEPT,OUTSTANDING_AFTER_EIR_UWI,GROSS_LOAN_RPSIA,M_BNM_BALANCE2,EAD_PROPORTIONAL_MYR,EXPECTED_LOSS_MYR,RWA_AFTER_RISK_MIT_MYR,PRODUCT_GROUP_APC,ASSET_CLASS,P2_RWA_MIL,M_BNM_BALANCE_MIL,EAD_PROPORTIONAL_MYR_MIL,EXPECTED_LOSS_MYR_MIL,RWA_AFTER_RISK_MIT_MYR_MIL,OUTSTANDING_AFTER_EIR_UWI_MIL,GROSS_LOAN_RPSIA_MIL
1221,10013113,CHIN PIK YOON,Y,01. LOAN,640010002026,RTO,Others,3.0,HZK04,HZK04,Rent-To-Own v4,0.0,6.0,9.0,R3,0,M51,0.0,N,N,0,0,0,-9999,B03,0.0,0.0,0.0,578000.0,765.3,106126.93,01. Corporate Banking,RETAIL (AIRB),0.0,0.0,0.58,7.65e-04,0.11,0.0,0.0


In [166]:
#product n cust
Python_prod_cus = Python_prod.groupby(['M_CUS_NO'])[['M_BNM_BALANCE2']].sum().reset_index().merge(Python_both,on='M_CUS_NO',how='left')

In [169]:
Python_prod_cus['DIFF'] = Python_prod_cus['M_BNM_BALANCE2_x']-Python_prod_cus['M_BNM_BALANCE2_y']

In [170]:
Python_prod_cus

,M_CUS_NO,M_BNM_BALANCE2_x,M_BNM_BALANCE2_y,EAD_PROPORTIONAL_MYR,EXPECTED_LOSS_MYR,RWA_AFTER_RISK_MIT_MYR,SUM(OUTSTANDING),EAD,EXPECTED_LOSS,RWA_AFTER_SCALING,_merge,NAME,Teradata_Tag,DIFF
0,10000083,15050.92,15050.92,6.01e+05,2323.15,217969.78,15050.92,6.01e+05,2323.14,217969.78,both,MAK HAUR JIUNN,Y,0.00
1,10013113,0.00,15264.48,5.93e+05,765.30,121391.41,15264.48,5.93e+05,765.30,121391.41,both,CHIN PIK YOON,Y,-15264.48
2,10014267,10091.20,19860.00,8.13e+05,1074.04,146906.60,19860.00,8.13e+05,1074.04,146906.61,both,AHMAD ZAKRI BIN MD SALLEH,Y,-9768.80
3,10050207,82681.30,82681.30,6.12e+05,810.55,112402.97,82681.30,6.12e+05,810.56,112402.98,both,KOAY KENG YANG,Y,0.00
4,10053442,99893.20,99893.20,5.91e+05,2285.24,214413.42,99893.20,5.91e+05,2285.24,214413.43,both,NG SHU YU,Y,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5667,9977772,349924.53,478074.38,7.98e+05,4252.78,375187.97,478074.06,7.98e+05,4252.79,375187.97,both,ZURILAN BIN MAMAT,Y,-128149.85
5668,9980598,0.00,89951.88,6.69e+05,2238.87,300014.99,89951.88,6.69e+05,2238.87,300014.99,both,STAFF,Y,-89951.88
5669,9986472,126651.71,126651.71,5.71e+05,437.63,71966.89,126651.71,5.71e+05,437.63,71966.88,both,CHONG RAU YIT,Y,0.00
5670,9997708,384524.70,384524.70,1.03e+06,3992.12,374561.68,384524.70,1.03e+06,3992.12,374561.68,both,AHMAD NADZRI BIN ISMAIL,Y,0.00


In [171]:
Python.iloc[np.where(Python.M_CUS_NO=='10013113')]

,DATA_SOURCE,M_ACCOUNT_NO,M_CUS_NO,LEVEL_6_ORI,LEVEL_6,M_SUB_MARKET_SEGMENT_CD,M_PRODUCT_HIERARCHY_CD,M_SOURCE_PROD_TYPE_CD,SOURCE_PROD_TYPE_DSC,M_MTHS_IN_ARREARS,APPROACH,SECTOR_CATEGORY,PD_SEGMENT,RSME_INCLUSION_INDICATOR,RS_INCLUSION_INDICATOR,REPORT_INDICATOR,F_RPSIA,RPSIA_THIRD_PARTY,IMPAIRED_FLAG,F_NCIF_CUST,F_NCIF_ACCT,SECURITY_TYPE_CD,M_FINANCING_CONCEPT,OUTSTANDING_AFTER_EIR_UWI,GROSS_LOAN_RPSIA,M_BNM_BALANCE2,EAD_PROPORTIONAL_MYR,EXPECTED_LOSS_MYR,RWA_AFTER_RISK_MIT_MYR,PRODUCT_GROUP_APC,ASSET_CLASS,P2_RWA_MIL,M_BNM_BALANCE_MIL,EAD_PROPORTIONAL_MYR_MIL,EXPECTED_LOSS_MYR_MIL,RWA_AFTER_RISK_MIT_MYR_MIL,OUTSTANDING_AFTER_EIR_UWI_MIL,GROSS_LOAN_RPSIA_MIL
520593,01. LOAN,462432121676,10013113,Salary,Salary,3.0,H84,H84,GLC Salary Financing-I,0.0,2.0,9.0,0,0,0,0.0,N,N,0,0,0,-9999,A02,0.0,0.0,15264.48,15264.48,0.0,15264.48,11. Salary,RETAIL (SA),0.0,0.02,0.02,0.00e+00,0.02,0.0,0.0
758854,01. LOAN,640010002026,10013113,RTO,Others,3.0,HZK04,HZK04,Rent-To-Own v4,0.0,6.0,9.0,R3,0,M51,0.0,N,N,0,0,0,-9999,B03,0.0,0.0,0.00,578000.00,765.3,106126.93,01. Corporate Banking,RETAIL (AIRB),0.0,0.00,0.58,7.65e-04,0.11,0.0,0.0


In [168]:
sum(Python_prod.M_BNM_BALANCE2), sum(Python_prod_cus.M_BNM_BALANCE2_x), sum(Python_prod_cus.M_BNM_BALANCE2_y)

(44473866954.029976, 44473866954.03014, 44665057340.020134)

In [141]:
Python_prod_cus['Dif_BNM'] = Python_prod_cus['M_BNM_BALANCE2']-Python_prod_cus['SUM(OUTSTANDING)']
Python_prod_cus['Dif_EAD'] = Python_prod_cus['EAD_PROPORTIONAL_MYR']-Python_prod_cus['EAD']
Python_prod_cus['Dif_EL'] = Python_prod_cus['EXPECTED_LOSS_MYR']-Python_prod_cus['EXPECTED_LOSS']
Python_prod_cus['Dif_RWA'] = Python_prod_cus['RWA_AFTER_RISK_MIT_MYR']-Python_prod_cus['RWA_AFTER_SCALING']


In [143]:
Python_prod_cus = Python_prod_cus.sort_values(by='Dif_BNM', ascending=False)

In [145]:
#export
writer = pd.ExcelWriter(r'D:\Users\80005701\Desktop\IA Moodys VS DWH\01. IA_CB.xlsx',engine='xlsxwriter')

Python_prod.to_excel(writer, sheet_name='CB Acc Level'+str(date), index = False)
Python_prod_cus.to_excel(writer, sheet_name='CB Cust Level'+str(date), index = False)

#b.iloc[np.where(b._merge=="left_only")].to_excel(writer, sheet_name='Moodys cust'+str(date), index = False)
#Moodys_left.to_excel(writer, sheet_name='Moodys acc'+str(date), index = False)

writer.save()

In [21]:
Comb_cek_P = both_P.merge(both_M[['COUNTERPARTY_ORIGINAL','SUM(OUTSTANDING)','EAD','EXPECTED_LOSS','RWA_AFTER_SCALING']].rename(columns={'COUNTERPARTY_ORIGINAL':'M_CUS_NO'}), on='M_CUS_NO',how='left')

Comb_cek_M = both_M.merge(both_P[['M_CUS_NO','M_BNM_BALANCE2','EAD_PROPORTIONAL_MYR','EXPECTED_LOSS_MYR','RWA_AFTER_RISK_MIT_MYR']].rename(columns={'M_CUS_NO':'COUNTERPARTY_ORIGINAL'}), on='COUNTERPARTY_ORIGINAL',how='left')

In [57]:
Comb_cek_P['Cek_dif_BNM'] =  Comb_cek_P.M_BNM_BALANCE2 - Comb_cek_P['SUM(OUTSTANDING)']

In [88]:
#Moodys_both = Moodys2.iloc[np.where(Moodys2._merge=='both')][['COUNTERPARTY_ORIGINAL','NAME','Teradata_Tag']].merge(Moodys, on='COUNTERPARTY_ORIGINAL',how='left')
#Moodys_both_prod = Moodys_both.iloc[np.where(Moodys_both['Product Group']=='01. Corporate Banking')]
